In [ ]:
# This is the chatbot

In [ ]:
import websockets
import json
import asyncio

BOT_TOKEN = "5a3556bb2de44a73ab2e5643cb633a6c"
THREAD_ID = "default"

async def chatbot():
    uri = f"ws://localhost:8000/ws/chat?thread_id={THREAD_ID}"
    messages = []
    async with websockets.connect(uri, extra_headers={"Cookie": f"Authorization=Bearer {BOT_TOKEN}"}) as ws:
        while True:
            try:
                raw_data = await asyncio.wait_for(ws.recv(), timeout=0.5)
                data = json.loads(raw_data)
                if data.get("replay", False):
                    messages.append(data)
                    continue
                if data["message"] == "break":
                    break
                if data["sender"] == "datAI":
                    # personal messages
                    continue
                if data.get("meta", False):
                    continue
            except asyncio.TimeoutError:
                # not sure why I'm doing this, it felt important to have an "infinite" loop...
                continue
            # Valid message to parse and likely respond to?
            await ws.send(json.dumps({"message": f"I'm a bot, and I saw your message ({data['message']})"}))

await chatbot()